# Visualization

In [ ]:
#|default_exp doc

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export 

from IPython.display import Image, display, HTML,SVG,display_pretty
from stringdale.core import jinja_render
from docstring_parser import parse
import inspect
from deepmerge import always_merger
from nbdev.doclinks import NbdevLookup


## pprinting yaml

In [ ]:
#| export
import yaml
from pygments import highlight
from pygments.lexers import YamlLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML, display
from pathlib import Path

In [ ]:
#| export
def pprint_yaml(yaml_obj, style=None):
    if isinstance(yaml_obj, str):
        yaml_str = yaml_obj
    elif isinstance(yaml_obj, Path):
        yaml_str = yaml_obj.read_text() 
    else:
        yaml_str = yaml.dump(yaml_obj, sort_keys=False)
    
    # Generate a unique class name based on the style
    class_name = f'highlight_{style if style else "default"}'
    if style is None:
        formatter = HtmlFormatter(cssclass=class_name)
    else:
        formatter = HtmlFormatter(style=style, cssclass=class_name)
    
    # Use the unique class name in the style definitions
    style_defs = formatter.get_style_defs(f'.{class_name}')
    highlighted = highlight(yaml_str, YamlLexer(), formatter)
    display(HTML(f"<style>{style_defs}</style>{highlighted}"))


In [ ]:
obj = {"foo": "bar", "hey": ["h","o"]}
pprint_yaml(obj, style='native')

In [ ]:
pprint_yaml(obj)

## Get object documentation data

In [ ]:
from stringdale import Define,V,DiagramSchema,Diagram, Condition,JsonRenderer

In [ ]:
#| export
def clean_sig_data(obj):
    sig = inspect.signature(obj)
    clean_data = {'params':{},
     'signature':str(sig)
    }
    for key, param in sig.parameters.items():
        p_dat = {}
        if param.annotation != inspect._empty:
            p_dat['type'] = param.annotation
        if param.default != inspect._empty:
            p_dat['default'] = param.default
        clean_data['params'][key] = p_dat
    if sig.return_annotation != inspect._empty:
        clean_data['returns'] = {
            'type': sig.return_annotation,
        }
    return clean_data
    

In [ ]:
clean_sig_data(V)

{'params': {'name': {'type': str},
  'func': {'type': typing.Callable, 'default': None},
  'inputs': {'type': typing.Any, 'default': None},
  'outputs': {'default': None},
  'is_break': {'type': bool, 'default': False},
  'for_each': {'type': typing.Optional[typing.List[str]], 'default': None},
  'filter': {'type': bool, 'default': False},
  'flat': {'type': bool, 'default': False},
  'as_start': {'type': bool, 'default': False},
  'as_end': {'type': bool, 'default': False}},
 'signature': '(name: str, func: Callable = None, inputs: Any = None, outputs=None, is_break: bool = False, for_each: Optional[List[str]] = None, filter: bool = False, flat: bool = False, as_start: bool = False, as_end: bool = False) -> None',
 'returns': {'type': None}}

In [ ]:
clean_sig_data(JsonRenderer)

{'params': {'json_obj': {}, 'kwargs': {}}, 'signature': '(json_obj, **kwargs)'}

In [ ]:
clean_sig_data(JsonRenderer.__call__)

{'params': {'self': {}, 'kwargs': {}}, 'signature': '(self, **kwargs)'}

In [ ]:
#| export
def clean_doc_data(obj):
    doc = parse(obj.__doc__)
    clean_data = {
        'description': doc.description,
        'params':{},
        'param_order':[p.arg_name for p in doc.params]
    }
    for p in doc.params:
        p_dat = {'name':p.arg_name}
        if p.type_name != inspect._empty:
            p_dat['type'] = p.type_name
        if p.default != inspect._empty:
            p_dat['default'] = p.default
        if p.description != inspect._empty:
            p_dat['description'] = p.description
        clean_data['params'][p.arg_name] = p_dat
        
    if doc.returns:
        r_dat = {}
        if doc.returns.type_name != inspect._empty:
            r_dat['type'] = doc.returns.type_name
        if doc.returns.description != inspect._empty:
            r_dat['description'] = doc.returns.description
        clean_data['returns'] = r_dat
    return clean_data

In [ ]:
clean_doc_data(V)

{'description': 'Add a vertex (node) to the current diagram.\n',
 'params': {'name': {'name': 'name',
   'type': 'str',
   'default': None,
   'description': 'Name of the node'},
  'func': {'name': 'func',
   'type': 'callable',
   'default': None,
   'description': 'Function to execute at this node. If None, node acts as a passthrough'},
  'inputs': {'name': 'inputs',
   'type': 'List[str]',
   'default': None,
   'description': 'List of input edge descriptors. Each descriptor can be either:\n- A string in format "source_node.source_port->target_port" \n- A tuple (edge_descriptor, condition_func) for conditional edges'},
  'outputs': {'name': 'outputs',
   'type': 'List[str]',
   'default': None,
   'description': 'List of output edge descriptors. Each descriptor can be either:\n- A string in format "source_port->target_node.target_port"\n- A tuple (edge_descriptor, condition_func) for conditional edges'},
  'is_break': {'name': 'is_break',
   'type': 'bool',
   'default': None,
   'd

In [ ]:
#| export
def get_methods(cls,include=None):
    if include is None:
        include = [method for method in dir(cls) if callable(getattr(cls, method)) and not method.startswith("_")] #+ ['__call__']
    included_methods = [method for method in include if callable(getattr(cls, method,None))]
    return included_methods


In [ ]:
get_methods(DiagramSchema)

['draw',
 'factor_diagram',
 'get_input_only_state_keys',
 'has_breakpoints',
 'post_def']

In [ ]:
#| export
lookup_table = NbdevLookup()

def get_source_link(obj):
    try:
        return lookup_table[obj.__qualname__][2]
    except:
        return None

In [ ]:
from stringdale.tools import google_search

In [ ]:
clean_sig_data(google_search.func)

{'params': {'q': {'type': str},
  'location': {'type': str, 'default': 'Austin, Texas'},
  'engine': {'type': str, 'default': 'google_scholar'}},
 'signature': "(q: str, location: str = 'Austin, Texas', engine: str = 'google_scholar')"}

In [ ]:
#| export
def escape_dunder(name):
    return name.replace('__','\\_\\_')


def clean_obj_data(obj):
    merged_data = always_merger.merge(clean_doc_data(obj),clean_sig_data(obj))
    merged_data['name'] = escape_dunder(obj.__name__)
    merged_data['clean_name'] = obj.__name__
    merged_data['source'] = get_source_link(obj)
    merged_data['params'] = [merged_data['params'][name] for name in merged_data['param_order']]
    return merged_data

In [ ]:
clean_obj_data(V)

{'description': 'Add a vertex (node) to the current diagram.\n',
 'params': [{'name': 'name',
   'type': str,
   'default': None,
   'description': 'Name of the node'},
  {'name': 'func',
   'type': typing.Callable,
   'default': None,
   'description': 'Function to execute at this node. If None, node acts as a passthrough'},
  {'name': 'inputs',
   'type': typing.Any,
   'default': None,
   'description': 'List of input edge descriptors. Each descriptor can be either:\n- A string in format "source_node.source_port->target_port" \n- A tuple (edge_descriptor, condition_func) for conditional edges'},
  {'name': 'outputs',
   'type': 'List[str]',
   'default': None,
   'description': 'List of output edge descriptors. Each descriptor can be either:\n- A string in format "source_port->target_node.target_port"\n- A tuple (edge_descriptor, condition_func) for conditional edges'},
  {'name': 'is_break',
   'type': bool,
   'default': False,
   'description': 'If True, execution will pause at t

In [ ]:
#| export

def clean_class(cls,methods=None):
    methods = get_methods(cls,methods)
    methods_data = [clean_obj_data(getattr(cls,method)) for method in methods]
    return {
        **clean_obj_data(cls),
        'methods':methods_data
    }

In [ ]:
clean_class(JsonRenderer,methods=['__call__'])

{'description': 'A class for rendering JSON objects with nested jinja2 templates.\n\nAllows setting template variables both during init and when calling the object.',
 'params': [{'name': 'json_obj',
   'type': None,
   'default': None,
   'description': 'A JSON object to render.'},
  {'name': '**kwargs',
   'type': None,
   'default': None,
   'description': 'Context variables to use in the rendering.'}],
 'param_order': ['json_obj', '**kwargs'],
 'returns': {'type': None, 'description': 'A rendered JSON object.'},
 'signature': '(json_obj, **kwargs)',
 'name': 'JsonRenderer',
 'clean_name': 'JsonRenderer',
 'source': 'https://github.com/DeanLight/stringdale/blob/main/stringdale/utils.py',
 'methods': [{'description': None,
   'params': [],
   'param_order': [],
   'signature': '(self, **kwargs)',
   'name': '\\_\\_call\\_\\_',
   'clean_name': '__call__',
   'source': 'https://github.com/DeanLight/stringdale/blob/main/stringdale/utils.py'}]}

## Rendering to markdown

In [ ]:
#| export
from IPython.display import Markdown,display
from stringdale.core import jinja_render

### Functions

In [ ]:
#| export
function_doc_template = """
### {{name}}

<p align="right"> <a href="{{source}}">source</a> </p>

> **Signature:** `{{clean_name}}{{signature}}`

{{description}}

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
{%  for param in params -%}
| {{param.name}} | {{param.type}} | {{param.default}} | {{param.description | replace('\n', '') | safe }} |
{%  endfor %}
{%- if returns -%}
| :Returns: | {{returns.type}} | - | {{returns.description | replace('\n', '<br>') | safe }} |
{%  endif %}

"""

In [ ]:
func_data = clean_obj_data(V)
# print(jinja_render(function_doc_template,func_data))

In [ ]:
display(Markdown(jinja_render(function_doc_template,func_data)))


### V

<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/diagrams_old.py">source</a> </p>

> **Signature:** `V(name: str, func: Callable = None, inputs: Any = None, outputs=None, is_break: bool = False, for_each: Optional[List[str]] = None, filter: bool = False, flat: bool = False, as_start: bool = False, as_end: bool = False) -> None`

Add a vertex (node) to the current diagram.


| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| name | <class 'str'> | None | Name of the node |
| func | typing.Callable | None | Function to execute at this node. If None, node acts as a passthrough |
| inputs | typing.Any | None | List of input edge descriptors. Each descriptor can be either:- A string in format "source_node.source_port->target_port" - A tuple (edge_descriptor, condition_func) for conditional edges |
| outputs | List[str] | None | List of output edge descriptors. Each descriptor can be either:- A string in format "source_port->target_node.target_port"- A tuple (edge_descriptor, condition_func) for conditional edges |
| is_break | <class 'bool'> | False | If True, execution will pause at this node. Not allowed in flow scopes |
| for_each | typing.Optional[typing.List[str]] | None | List of input keys to iterate over. Used for batching operations in Flow diagrams.If provided, the node will be executed once for each product of items in the input list.This for each keys must get iteratbles from the input edges. |
| filter | <class 'bool'> | False | Used for batching operations in Flow diagrams. If True, falsy node outputs will be filtered out. Cannot be used with flat=True |
| flat | <class 'bool'> | False | Used for batching operations in Flow diagrams.If True, node output lists will be flattened into a single list.Cannot be used with filter=True |
| as_start | <class 'bool'> | False | If True, marks this node as the diagram's start node |
| as_end | <class 'bool'> | False | If True, marks this node as the diagram's end node |
| :Returns: | None | - | Name of the created node |



### Classes

In [ ]:
#| export

class_doc_template = """
### {{name}}
<p align="right"> <a href="{{source}}">source</a> </p>

> **Signature:** `{{clean_name}}{{signature}}`

{{description}}

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
{%  for param in params -%}
| {{param.name}} | {{param.type}} | {{param.default}} | {{param.description | replace('\n', '') | safe }} |
{%  endfor %}
{%- if returns -%}
| :Returns: | {{returns.type}} | - | {{returns.description | replace('\n', '<br>') | safe }} |
{%  endif %}

{% for method in methods %}
#### {{ method.name }}

{{method.description}}
> **Signature:** `{{clean_name}}.{{method.clean_name}}{{method.signature}}`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
{%  for param in method.params -%}
| {{param.name}} | {{param.type}} | {{param.default}} | {{param.description | replace('\n', '') | safe }} |
{%  endfor -%}
{% if returns -%}
| :Returns: | {{returns.type}} | - | {{returns.description | replace('\n', '<br>') | safe }} |
{%  endif %}

{% endfor %}
"""

In [ ]:
class_data = clean_class(JsonRenderer)
class_data


{'description': 'A class for rendering JSON objects with nested jinja2 templates.\n\nAllows setting template variables both during init and when calling the object.',
 'params': [{'name': 'json_obj',
   'type': None,
   'default': None,
   'description': 'A JSON object to render.'},
  {'name': '**kwargs',
   'type': None,
   'default': None,
   'description': 'Context variables to use in the rendering.'}],
 'param_order': ['json_obj', '**kwargs'],
 'returns': {'type': None, 'description': 'A rendered JSON object.'},
 'signature': '(json_obj, **kwargs)',
 'name': 'JsonRenderer',
 'clean_name': 'JsonRenderer',
 'source': 'https://github.com/DeanLight/stringdale/blob/main/stringdale/utils.py',
 'methods': []}

In [ ]:
print(jinja_render(class_doc_template,params=class_data))


### JsonRenderer
<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/utils.py">source</a> </p>

> **Signature:** `JsonRenderer(json_obj, **kwargs)`

A class for rendering JSON objects with nested jinja2 templates.

Allows setting template variables both during init and when calling the object.

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| json_obj | None | None | A JSON object to render. |
| **kwargs | None | None | Context variables to use in the rendering. |
| :Returns: | None | - | A rendered JSON object. |





In [ ]:
display(Markdown(jinja_render(class_doc_template,params=class_data)))


### JsonRenderer
<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/utils.py">source</a> </p>

> **Signature:** `JsonRenderer(json_obj, **kwargs)`

A class for rendering JSON objects with nested jinja2 templates.

Allows setting template variables both during init and when calling the object.

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| json_obj | None | None | A JSON object to render. |
| **kwargs | None | None | Context variables to use in the rendering. |
| :Returns: | None | - | A rendered JSON object. |




### All together

In [ ]:
#| export
def show_doc(obj,methods=None):
    if inspect.isfunction(obj):
        data = clean_obj_data(obj)
        md = jinja_render(function_doc_template,data)
        
    elif inspect.isclass(obj):
        data = clean_class(obj,methods=methods)
        md = jinja_render(class_doc_template,data)

    else:
        raise ValueError(f"Unsupported object type: {type(obj)}")
    
    display(Markdown(md))

In [ ]:
show_doc(V)
show_doc(JsonRenderer,methods=['__call__','__str__'])



### V

<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/diagrams_old.py">source</a> </p>

> **Signature:** `V(name: str, func: Callable = None, inputs: Any = None, outputs=None, is_break: bool = False, for_each: Optional[List[str]] = None, filter: bool = False, flat: bool = False, as_start: bool = False, as_end: bool = False) -> None`

Add a vertex (node) to the current diagram.


| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| name | <class 'str'> | None | Name of the node |
| func | typing.Callable | None | Function to execute at this node. If None, node acts as a passthrough |
| inputs | typing.Any | None | List of input edge descriptors. Each descriptor can be either:- A string in format "source_node.source_port->target_port" - A tuple (edge_descriptor, condition_func) for conditional edges |
| outputs | List[str] | None | List of output edge descriptors. Each descriptor can be either:- A string in format "source_port->target_node.target_port"- A tuple (edge_descriptor, condition_func) for conditional edges |
| is_break | <class 'bool'> | False | If True, execution will pause at this node. Not allowed in flow scopes |
| for_each | typing.Optional[typing.List[str]] | None | List of input keys to iterate over. Used for batching operations in Flow diagrams.If provided, the node will be executed once for each product of items in the input list.This for each keys must get iteratbles from the input edges. |
| filter | <class 'bool'> | False | Used for batching operations in Flow diagrams. If True, falsy node outputs will be filtered out. Cannot be used with flat=True |
| flat | <class 'bool'> | False | Used for batching operations in Flow diagrams.If True, node output lists will be flattened into a single list.Cannot be used with filter=True |
| as_start | <class 'bool'> | False | If True, marks this node as the diagram's start node |
| as_end | <class 'bool'> | False | If True, marks this node as the diagram's end node |
| :Returns: | None | - | Name of the created node |




### JsonRenderer
<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/utils.py">source</a> </p>

> **Signature:** `JsonRenderer(json_obj, **kwargs)`

A class for rendering JSON objects with nested jinja2 templates.

Allows setting template variables both during init and when calling the object.

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| json_obj | None | None | A JSON object to render. |
| **kwargs | None | None | Context variables to use in the rendering. |
| :Returns: | None | - | A rendered JSON object. |



#### \_\_call\_\_

None
> **Signature:** `JsonRenderer.__call__(self, **kwargs)`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| :Returns: | None | - | A rendered JSON object. |



#### \_\_str\_\_

None
> **Signature:** `JsonRenderer.__str__(self)`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| :Returns: | None | - | A rendered JSON object. |




# Export

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     